#### Let's validate the model using DRUM

In [1]:
!drum validation --code-dir ../src/custom_model --input ../data/boston_housing_test.csv --target-type regression

Traceback (most recent call last):
  File "/Users/timothy.whittaker/python_envs/odsc/bin/drum", line 6, in <module>
    main()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/datarobot_drum/drum/main.py", line 96, in main
    CMRunner(runtime).run()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/datarobot_drum/drum/drum.py", line 289, in run
    self._run_fit_and_predictions_pipelines_in_mlpiper()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/datarobot_drum/drum/drum.py", line 545, in _run_fit_and_predictions_pipelines_in_mlpiper
    _pipeline_executor.run_pipeline(cleanup=False)
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/mlpiper/pipeline/executor.py", line 261, in run_pipeline
    self._run_pipeline()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/mlpiper/pipeline/executor.py", line 251, in _run_pipeline
    self._dag.run_connec

Traceback (most recent call last):
  File "/Users/timothy.whittaker/python_envs/odsc/bin/drum", line 6, in <module>
    main()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/datarobot_drum/drum/main.py", line 96, in main
    CMRunner(runtime).run()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/datarobot_drum/drum/drum.py", line 289, in run
    self._run_fit_and_predictions_pipelines_in_mlpiper()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/datarobot_drum/drum/drum.py", line 545, in _run_fit_and_predictions_pipelines_in_mlpiper
    _pipeline_executor.run_pipeline(cleanup=False)
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/mlpiper/pipeline/executor.py", line 261, in run_pipeline
    self._run_pipeline()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/mlpiper/pipeline/executor.py", line 251, in _run_pipeline
    self._dag.run_connec

Traceback (most recent call last):
  File "/Users/timothy.whittaker/python_envs/odsc/bin/drum", line 6, in <module>
    main()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/datarobot_drum/drum/main.py", line 96, in main
    CMRunner(runtime).run()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/datarobot_drum/drum/drum.py", line 289, in run
    self._run_fit_and_predictions_pipelines_in_mlpiper()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/datarobot_drum/drum/drum.py", line 545, in _run_fit_and_predictions_pipelines_in_mlpiper
    _pipeline_executor.run_pipeline(cleanup=False)
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/mlpiper/pipeline/executor.py", line 261, in run_pipeline
    self._run_pipeline()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/mlpiper/pipeline/executor.py", line 251, in _run_pipeline
    self._dag.run_connec

Traceback (most recent call last):
  File "/Users/timothy.whittaker/python_envs/odsc/bin/drum", line 6, in <module>
    main()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/datarobot_drum/drum/main.py", line 96, in main
    CMRunner(runtime).run()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/datarobot_drum/drum/drum.py", line 289, in run
    self._run_fit_and_predictions_pipelines_in_mlpiper()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/datarobot_drum/drum/drum.py", line 545, in _run_fit_and_predictions_pipelines_in_mlpiper
    _pipeline_executor.run_pipeline(cleanup=False)
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/mlpiper/pipeline/executor.py", line 261, in run_pipeline
    self._run_pipeline()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/mlpiper/pipeline/executor.py", line 251, in _run_pipeline
    self._dag.run_connec

Traceback (most recent call last):
  File "/Users/timothy.whittaker/python_envs/odsc/bin/drum", line 6, in <module>
    main()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/datarobot_drum/drum/main.py", line 96, in main
    CMRunner(runtime).run()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/datarobot_drum/drum/drum.py", line 289, in run
    self._run_fit_and_predictions_pipelines_in_mlpiper()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/datarobot_drum/drum/drum.py", line 541, in _run_fit_and_predictions_pipelines_in_mlpiper
    _pipeline_executor.init_pipeline()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/mlpiper/pipeline/executor.py", line 257, in init_pipeline
    self._init_pipeline()
  File "/Users/timothy.whittaker/python_envs/odsc/lib/python3.7/site-packages/mlpiper/pipeline/executor.py", line 204, in _init_pipeline
    self._init_ml_engine(self.pip

As you can see, we are getting an error that `ValueError: Failure when making predictions: Input contains NaN, infinity or a value too large for dtype('float32').`

Basically, the model is not robust enough to handle null values and while our data does not include any null values, DRUM is checking automatically. To solve this, we are going to add a `custom.py` script within the `custom_model` folder which will fill null values if they are encountered.

When models don't fit the drop in pattern, we can create custom hooks to hook the model up to drum.

In [2]:
!cat ../src/custom.py

import pandas as pd
from typing import List, Optional, Any, Dict

# """
# Custom hooks for prediction
# ---------------------------
# If drum's standard assumptions are incorrect for your model, **drum** supports several hooks
# for custom inference code.
# """
# def init(code_dir : Optional[str], **kwargs) -> None:
#     """

#     Parameters
#     ----------
#     code_dir : code folder passed in the `--code_dir` parameter
#     kwargs : future proofing
#     """


# def load_model(code_dir: str) -> Any:
#     """
#     Can be used to load supported models if your model has multiple artifacts, or for loading
#     models that **drum** does not natively support

#     Parameters
#     ----------
#     code_dir : is the directory where model artifact and additional code are provided, passed in

#     Returns
#     -------
#     If used, this hook must return a non-None value
#     """
#     return CustomModel(code_dir)


def transform(data: pd.DataFrame, model: Any) -> pd.DataFrame:
  

In the code above, we have only created a transform hook which could handle preprocessing including missing value fill in.  

In [3]:
!cp ../src/custom.py ../src/custom_model/custom.py

In [5]:
!drum validation --code-dir ../src/custom_model --input ../data/boston_housing_test.csv --target-type regression

    Predictions
0        26.210
1        21.885
2        34.845
3        34.235
4        34.015
5        26.795
6        20.370
7        25.035
8        20.850
9        20.100
10       21.075
11       19.030
12       21.695
13       19.310
14       17.995
15       19.100
16       21.035
17       19.510
18       19.400
19       18.380
    Predictions
0        26.210
1        22.140
2        34.930
3        34.735
4        35.200
5        26.795
6        20.885
7        24.655
8        18.160
9        18.690
10       16.365
11       19.610
12       21.650
13       20.010
14       18.400
15       19.685
16       22.185
17       18.350
18       19.790
19       18.740
    Predictions
0        26.210
1        21.285
2        34.800
3        34.735
4        35.200
5        26.890
6        21.595
7        25.035
8        18.205
9        18.785
10       16.335
11       20.290
12       21.605
13       19.720
14       19.610
15       19.465
16       22.105
17       18.725
18       21.545
19      

After moving the `custom.py` file inside the `custom_model` folder, we see that the results come back normally.